In [1]:
%matplotlib inline
from pathlib import *
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import os
import cv2
import sys 

In [2]:
sys.path.append('../../fastai/')

In [3]:
from fastai.conv_learner import *

In [139]:
path = list(Path('../data/ds_bowl_2018/').iterdir())

In [140]:
path

[PosixPath('../data/ds_bowl_2018/stage2_test_final'),
 PosixPath('../data/ds_bowl_2018/stage1_train_labels.csv'),
 PosixPath('../data/ds_bowl_2018/stage1_train'),
 PosixPath('../data/ds_bowl_2018/stage1_solution.csv')]

In [10]:
trn_dirs = list(path[2].iterdir()) # stage1_train

In [12]:
def run_length_decode(rle, H, W, fill_value=255):

    mask = np.zeros((H * W), np.uint8)
    rle = np.array([int(s) for s in rle.split(' ')]).reshape(-1, 2)
    for r in rle:
        start = r[0]-1
        end = start + r[1]
        mask[start : end] = fill_value
    mask = mask.reshape(W, H).T # H, W need to swap as transposing.
    return mask

In [13]:
stage1_solution = pd.read_csv(path[-1])
stage1_train_labels = pd.read_csv(path[1])

In [14]:
stage1_all_labels = pd.concat([stage1_train_labels, stage1_solution.iloc[:, :2]])

In [15]:
stage1_all_labels.head()

,ImageId,EncodedPixels
0,00071198d059ba7f5914a526d124d28e6d010c92466da2...,6908 1 7161 8 7417 8 7672 9 7928 9 8184 9 8440...
1,00071198d059ba7f5914a526d124d28e6d010c92466da2...,36269 7 36523 11 36778 13 37033 15 37288 17 37...
2,00071198d059ba7f5914a526d124d28e6d010c92466da2...,19919 6 20174 8 20429 10 20685 11 20941 12 211...
3,00071198d059ba7f5914a526d124d28e6d010c92466da2...,18671 6 18926 8 19181 9 19436 10 19691 11 1994...
4,00071198d059ba7f5914a526d124d28e6d010c92466da2...,40158 3 40413 5 40669 5 40925 5 41182 3


### Simple Best Approach - 2 Channel Output: Mask and Contours

In [83]:
# creating mask files for stage 1 test files
for trn_dir in trn_dirs:
    dirname = trn_dir.name
    try:
        os.makedirs(trn_dir/'masks/', exist_ok=False) # create masks dir

        test_case = stage1_all_labels[stage1_all_labels.ImageId == dirname].reset_index()
        rles = test_case['EncodedPixels']
        H, W = open_image(list((trn_dir/'images').iterdir())[0]).shape[:-1]

        for i, rle in enumerate(rles):
            decoded = run_length_decode(rle, H, W, 255)
            plt.imsave((str(trn_dir/'masks') + f'/mask{i}.png'), decoded)
    except:pass

In [161]:
# creating one mask files for every image - target channel 1 
for trn_dir in trn_dirs:
    # create one mask for every image
    one_mask_fn = trn_dir/'one_mask.png'
    mask_paths = list((trn_dir/'masks').iterdir())

    combined_masks = 0
    for mask_path in mask_paths:
        mask = cv2.imread(str(mask_path), cv2.IMREAD_GRAYSCALE)
        combined_masks += (mask - 30)/185 
    combined_masks = np.clip(combined_masks, 0, 255)

    #if 'one_mask.png' not in [o.name for o in list(dirpath.iterdir())]:
    plt.imsave(one_mask_fn, combined_masks)

In [127]:
# creating contour files for every image - target channel 2
def get_contoured_mask2d(mask):
    """For a single mask file get contour"""
    gray = cv2.cvtColor(mask, cv2.COLOR_BGR2GRAY)
    (t, binary) = cv2.threshold(gray, 0, 255, cv2.THRESH_BINARY)
    (_, contours, _) = cv2.findContours(binary.astype(np.uint8), 
                               cv2.RETR_EXTERNAL,cv2.CHAIN_APPROX_SIMPLE)
    contoured_mask = cv2.drawContours(mask, contours, -1, (0, 0, 255), 2)
    

#    contoured_mask = get_contoured_mask(mask)

    contoured_mask2d = ((contoured_mask[:, :, 0]==1)*1) + ((contoured_mask[:, :, 2]==255)*2)
    return contoured_mask2d

# append all contoured mask files
def get_all_contoured_masks(masks):
    contoured_masks = []
    for mask in masks:
        contoured_masks.append(get_contoured_mask2d(mask))

    all_contoured_masks = np.sum(np.array(contoured_masks), 0)
    all_contoured_masks = np.clip(all_contoured_masks, 0,2)
    return all_contoured_masks



for trn_dir in trn_dirs:
    # create one mask for every image
    cntr_mask_fn = trn_dir/'contour_mask.png'
    masks = [open_image(str(o)) for o in list((trn_dir/'masks').iterdir())]
    plt.imsave(cntr_mask_fn, (get_all_contoured_masks(masks) == 2)*255)

### Check

In [23]:
for trn_dir in trn_dirs:
    if open_image(str(trn_dir/'one_mask/one_mask.png')) is None:
        print(trn_dir.name)